# 批标准化

1. [它是什么？](#theory)
2. [它有什么优点？](#benefits)
3. [我们如何将它添加到网络中？](#implementation_1)
4. [来看它是如何工作的！](#demos)
5. [What are you hiding?](#implementation_2)

# 什么是批标准化？<a id='theory'></a>

　　批标准化概念来自于一篇论文 [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/pdf/1502.03167.pdf). 它的想法是, 将网络的各层输入进行标准化，而不是只对网络的输入层进行标准化。 它叫"批"标准化，是因为在训练中，我们使用当前的mini-batch的均值和方差对每层输入进行标准化。<br />
　　这样有什么用呢？我们知道对网络的输入层进行标准化处理有助于网络的学习（梯度下降会更快）。而神经网络有一系列层组成的，其中一层的输出会成为下一层的输入。这意味着我们可以将神经网络的任何一层当做小的网络的一层。<br />
　　举例，想像一个三层网络。不要将该网络想成由输入层，隐藏层和输出层组成，而是想成第一层的输出作为后两层网络的输入。这个两层网络由原始网络的第二层和第三层组成。另外，第二层的输出可以认为是最后一层单一网络的输入。<br />
　　当你这样想时，作为一系列神经网络相互嵌套（一层的输出为下一层的输入），然后很容易明白为什么对每层的输入进行标准化是有用的。就像对其它神经网络的输入层进行标准化，但你是对每层进行标准化操作（子网络）。<br />
　　这里还有些数学理论能帮助你明白为什么神经网络会学的更好。It helps combat what the authors call _internal covariate shift_. This discussion is best handled [in the paper](https://arxiv.org/pdf/1502.03167.pdf) and in [Deep Learning](http://www.deeplearningbook.org) a book you can read online written by Ian Goodfellow, Yoshua Bengio, and Aaron Courville. Specifically, check out the batch normalization section of [Chapter 8: Optimization for Training Deep Models](http://www.deeplearningbook.org/contents/optimization.html).

# 批标准化优点<a id="benefits"></a>

批标准化能优化网络训练。以下是它的几个优点：
1. **网络训练更快** – 由于在正向传播有额外的计算和反向传播中有|另外的参数要训练，导致每次训练迭代实际会变得很慢。但是，它会收敛的更快，所以整体上训练会更快；
2. **能使用更大的学习速率** – 为了网络能够收敛，梯度下降通常会使用更小的学习速率。当一个网络越深，在反向传播中它的梯度会变小，导致需要更多的迭代训练。使用批标准化允许我们使用更大的学习速率，将会加快网络训练的速度；
3. **使权重更容易初始化** – 权重初始化是很困难的，特别是在创建深层网络时。批标准化可以让我们选择权重初始化不用太谨慎； 
4. **更多激活函数可选择** – 在有些情况下，部分激活函数不适用。如Sigmoids的梯度消失，这意味着我们不能在很深的网络中使用它。ReLU也经常在训练期间消失，在那里停止学习，所以我们需要小心确认放入的数值范围。因为批标准化能调节每个进入激活函数的值，所以在深层网络中似乎不起作用的非线性激活函数，实际是可以用的；
5. **简化深层网络的创建** – 基于前面列出的四项，当使用批标准化时，更容易建立一个能快速训练的深层神经网络，并且能获得更好的训练结果。
6. **提供一点正则作用** – 批标准化会对网络添加一点噪声，比如在Inception模型中，批标准化和dropout是可以用的。但总体上，认为批标准化有一点额外的正则作用，可以允许你减少dropout，即降低网络中神经元丢弃的概率；
7. **总体上能获得更好结果** – 一些测试表明批标准化实际能提升训练结果。然而，这是能使网络训练的更快，所以你不该认为它是能使网络更好的方法。但是，由于它可以更快地训练网络，所以你可以更加快速迭代地不同网络设计。批标准化还可以建立更深的网络，通常表现很好。在构建网络时使用批标准化，会得到更好的结果。

### 批标准化的一些细节

　　使用 `tf.layers.batch_normalization` 可以创建批标准层，讲讲里面的细节。为了对值进行标准化，我们首先要找到 batch 数据的平均值。如果你看过代码，会发现找的并不是batch _inputs_ 的平均值，而是我们将其传递给非线性激活函数前，从任何特定神经元层出来的平均值，然后将其作为输入传递给下一层。将 $\mu_B$ 作为平均值，其计算公式为对和求平均：
$$
\mu_B \leftarrow \frac{1}{m}\sum_{i=1}^m x_i
$$
　　然后要计算方差，或者均方偏差，用 $\sigma_{B}^{2}$ 来表示。计算公式如下（减去均值为偏差，然后在平方，将所有结果相加）：
 $$
\sigma_{B}^{2} \leftarrow \frac{1}{m}\sum_{i=1}^m (x_i - \mu_B)^2
$$
　　一旦有了均值和方差，我们可以使用以下等式对值进行标准化。对于每个值，减去均值和除以标准差：
$$
\hat{x_i} \leftarrow \frac{x_i - \mu_B}{\sqrt{\sigma_{B}^{2} + \epsilon}}
$$
　　

　　以上，我们说"（几乎）标准差"，这是因为batch的真正标准差是通过 $\sqrt{\sigma_{B}^{2}}$ 计算的，但这个式子添加了误差项 $\epsilon$，在取平方根前。这个误差项可以为非常小的正的常数，这里使用 `0.001`。确保不会除以0，但会稍微增加每个batch的方差。为什么会增加方差？从统计学来看，即使我们在标准化一个 batch ，同样也在估计其分布。整个训练集，本身是网络想要的分布。总体方差高于从该分布取的样本方差，考虑到这一点所以对每个 batch 增加一点方差。
　　将标准化的值，叫做 $\hat{x_i}$。但不是直接使用它，而是乘以一个伽马值，$\gamma$，再加上一个 $\beta$。这两个参数 $\gamma$ 和 $\beta$ 都是通过网络学习的，用于缩放和平移标准化的值。它们像权重一样可以学习，在训练期间给予网络一些调整功能以此来逼近真实训练目标。


$$
y_i \leftarrow \gamma \hat{x_i} + \beta
$$

　　现在输出层有了最后 batch-normalized 的输出，就可以使用非线性函数如  sigmoid, tanh, ReLU, Leaky ReLU 等。在原始的 batch normalization paper中，提出在非线性函数后再执行标准化，但在实践中基本没这么做的。在 `NeuralNet`' 关于 `fully_connected` 的实现，数学计算都隐藏在下面这行，其中 `linear_output` :
```python
batch_normalized_output = tf.layers.batch_normalization(linear_output, training=self.is_training)
```

### Batch normalization without the `tf.layers` package
　　使用 [tf.layers.batch_normalization](https://www.tensorflow.org/api_docs/python/tf/layers/batch_normalization)来实现批标准化。你可以将 `NeuralNet` 类中的函数 `fully_connected` 替换为下面的代码，一样可以使用。

In [ ]:
def fully_connected(self, layer_in, initial_weights, activation_fn=None):
    '''
    创建一个标准的，全连接层。它的输入和输出由 `initial_weights` 的大小来定义，其初始权重值
    直接从相同参数中获得。如果 `self.use_batch_norm` 为True，则使用批标准化。
        
    :param layer_in: Tensor
        进入该层的张量，可以是网络的输入或是前一层的输出。
    :param initial_weights: NumPy array or Tensor
        该层权重的初始值， (784, 256) 表示创建一个输入为784个神经元，输出为256个神经元的神经元层。
    :param activation_fn: Callable or None (default None)
        对该层的输出使用非线性。如果为None，则该层不使用批标准化，不管 `self.use_batch_norm` 的值。
    '''
    if self.use_batch_norm and activation_fn:
        # 批标准化与往常一样使用权重，但不用加上偏差，这是因为在计算时有 gamma 和 beta 变量，
        # 起到了偏差作用，所以不需要偏差。
        weights = tf.Variable(initial_weights)
        linear_output = tf.matmul(layer_in, weights)
    
        # 输出的维度
        num_out_nodes = initial_weights.shape[-1]

        # 批标准化加上两个需要学习训练的变量，其中 gamma 用于缩放，beta 用于平移（类似偏差的作用）: 
        gamma = tf.Variable(tf.ones([num_out_nodes]))
        beta = tf.Variable(tf.zeros([num_out_nodes]))
    
        # 以下变量存储该层的均值和方差，假设它表示总体的分布。
        # 通过设置  `trainable=False`，告诉网络在反向传播期间不需要修改这些变量，而是自己为这些变量赋值
        pop_mean = tf.Variable(tf.zeros([num_out_nodes]), trainable=False)
        pop_variance = tf.Variable(tf.ones([num_out_nodes]), trainable=False)
        
        # 批标准需要一个小的常数值，避免除数为0
        epsilon = 1e-3

        def batch_norm_training():
            # 计算该层线性输出（输入*权重）的均值和方差，[0] 表示以第0轴来计算。
            batch_mean, batch_variance = tf.nn.moments(linear_output, [0])
            
            # 在训练时，计算训练数据的均值和方差的移动平均值，用于测试集验证模型。
            # Decay 通常设置小于1，tf.layers.batch_normalization 通常使用参数 "动量"  来完成，默认为0.99
            decay = 0.99
            train_mean = tf.assign(pop_mean, pop_mean * decay + batch_mean * (1 - decay))
            train_variance = tf.assign(pop_variance, pop_variance * decay + batch_variance * (1 - decay))
            
            # 这里' tf.control_dependencies' 文本告诉 TensorFlow 在计算 'tf.nn.batch_normalization' 层
            # 前需要计算  'train_mean'  和 'train_variance' 。这是必须的，因为这两个操作实际上并不在连接
            # linear_output和batch_normalization图层的图形中，所以TensorFlow会跳过它们。
            with tf.control_dependencies([train_mean, train_variance]):
                return tf.nn.batch_normalization(linear_output, batch_mean, batch_variance, beta, gamma, epsilon)
 
        def batch_norm_inference():
            # 在测试集验证模型时，使用估计的均值population mean和方差population variance
            return tf.nn.batch_normalization(linear_output, pop_mean, pop_variance, beta, gamma, epsilon)
        
        
        # 使用 `tf.cond` 
        # Use `tf.cond` as a sort of if-check. When self.is_training is True, TensorFlow will execute 
        # the operation returned from `batch_norm_training`; otherwise it will execute the graph
        # operation returned from `batch_norm_inference`.
        batch_normalized_output = tf.cond(self.is_training, batch_norm_training, batch_norm_inference)
            
        # Pass the batch-normalized layer output through the activation function.
        # The literature states there may be cases where you want to perform the batch normalization *after*
        # the activation function, but it is difficult to find any uses of that in practice.
        return activation_fn(batch_normalized_output)
    else:
        # When not using batch normalization, create a standard layer that multiplies
        # the inputs and weights, adds a bias, and optionally passes the result 
        # through an activation function.  
        weights = tf.Variable(initial_weights)
        biases = tf.Variable(tf.zeros([initial_weights.shape[-1]]))
        linear_output = tf.add(tf.matmul(layer_in, weights), biases)
        return linear_output if not activation_fn else activation_fn(linear_output)
